In [ ]:
from gs_quant.backtests.triggers import DateTriggerRequirements, DateTrigger
from gs_quant.backtests.actions import AddTradeAction
from gs_quant.backtests.core import ValuationFixingType
from gs_quant.backtests.strategy import Strategy
from gs_quant.backtests.predefined_asset_engine import PredefinedAssetEngine
from gs_quant.backtests.data_sources import DataManager
from gs_quant.instrument import FXOption
from gs_quant.data import DataFrequency

from random import random
import pandas as pd
from datetime import date, datetime, timedelta

from gs_quant.session import GsSession
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

In [ ]:
trigger_times = [datetime(2021, 1, 4, 14, 43, 0), 
                 datetime(2021, 1, 6, 14, 43, 0), 
                 datetime(2021, 1, 6, 14, 58, 0), 
                 datetime(2021, 1, 7, 13, 28, 0), 
                 datetime(2021, 1, 7, 14, 58, 0), 
                 datetime(2021, 1, 8, 13, 28, 0), 
                 datetime(2021, 1, 13, 13, 28, 0), 
                 datetime(2021, 1, 14, 13, 28, 0),
                 datetime(2021, 1, 15, 13, 28, 0),
                 datetime(2021, 1, 15, 14, 13, 0),
                 datetime(2021, 1, 15, 14, 58, 0),
                 datetime(2021, 1, 21, 13, 28, 0),
                 datetime(2021, 1, 22, 14, 43, 0),
                 datetime(2021, 1, 22, 14, 58, 0),
                 datetime(2021, 1, 26, 14, 58, 0),
                 datetime(2021, 1, 27, 13, 28, 0),
                 datetime(2021, 1, 27, 18, 58, 0),
                 datetime(2021, 1, 28, 13, 28, 0),
                 datetime(2021, 1, 28, 14, 58, 0),
                 datetime(2021, 1, 29, 13, 28, 0),
                 datetime(2021, 1, 29, 14, 58, 0),
                 datetime(2021, 2, 1, 14, 43, 0),
                 datetime(2021, 2, 1, 14, 58, 0),
                 datetime(2021, 2, 3, 14, 43, 0),
                 datetime(2021, 2, 3, 14, 58, 0),
                 datetime(2021, 2, 4, 13, 28, 0),
                 datetime(2021, 2, 4, 14, 58, 0),
                 datetime(2021, 2, 5, 13, 28, 0),
                 datetime(2021, 2, 10, 13, 28, 0),
                 datetime(2021, 2, 11, 13, 28, 0),
                 datetime(2021, 2, 12, 14, 58, 0),
                 datetime(2021, 2, 17, 13, 28, 0),
                 datetime(2021, 2, 17, 14, 13, 0),
                 datetime(2021, 2, 18, 13, 28, 0),
                 datetime(2021, 2, 19, 14, 43, 0),
                 datetime(2021, 2, 19, 14, 58, 0),
                 datetime(2021, 2, 23, 14, 58, 0),
                 datetime(2021, 2, 24, 14, 58, 0),
                 datetime(2021, 2, 25, 13, 28, 0),
                 datetime(2021, 2, 26, 13, 28, 0),
                 datetime(2021, 2, 26, 14, 58, 0),
                 datetime(2021, 3, 1, 14, 43, 0),
                 datetime(2021, 3, 1, 14, 58, 0),
                 datetime(2021, 3, 3, 14, 43, 0),
                 datetime(2021, 3, 3, 14, 58, 0),
                 datetime(2021, 3, 4, 13, 28, 0),
                 datetime(2021, 3, 4, 14, 58, 0),
                 datetime(2021, 3, 5, 13, 28, 0),
                 datetime(2021, 3, 10, 13, 28, 0)]

start_date, end_date = min(trigger_times), max(trigger_times)


In [ ]:
# Asset and Prices of asset
# the asset defined here could be anything as we are going to provide the valuation of that asset.
asset = FXOption(buy_sell='Buy', option_type='Put', pair='EURUSD',  notional_amount='10m', name='EURUSD')


# we need to provide the price of the asset in a data_manager

data_manager = DataManager()
index = pd.date_range(start_date, end_date.date() + timedelta(days=1), freq='T')
# Here I am constructing random prices for the asset
price = [100]
for i in range(1, len(index)):
    price.append(price[i-1] + (-1 if random() < 0.5 else 1))
prices = pd.Series(price, index=index)

eod_prices = prices.at_time('16:00').dropna()

data_manager.add_data_source(prices,
                             DataFrequency.REAL_TIME,
                             asset,
                             ValuationFixingType.PRICE)

data_manager.add_data_source(eod_prices,
                             DataFrequency.DAILY,
                             asset,
                             ValuationFixingType.PRICE)

In [ ]:
# define the triggers, actions and strategy - this is common across all backtest engine implementations

# the DateTrigger takes a list of dates when the an action should be taken
trig_req = DateTriggerRequirements(trigger_times)
# we are going to add our asset and hold it for 30 minutes
trigger = DateTrigger(trig_req, AddTradeAction(asset, pd.Timedelta(minutes=30)))
strategy = Strategy(None, trigger)


In [ ]:
# construct a PredefinedAssetEngine including our asset prices and then run the backtest
engine = PredefinedAssetEngine(data_manager)
bt = engine.run_backtest(strategy, start=start_date, end=end_date, frequency='D')

In [ ]:
# we can look at the performance of the backtest

bt.performance.plot()

In [ ]:
# or we can look at a trade ledge which shows the trades that were entered and their individual pnls
bt.trade_ledger()